# ONLY RUN THE NEXT THREE CELL's ONCE!

## Change the runtime Edit --> Notebook Settings --> Runtime Version --> 2025.07

In [ ]:
# Global variable to check if we're in Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    ! pip install numpy==1.26.4
    ### The runtime will crash/restart after this

In [ ]:
# Global variable to check if we're in Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:

  !pip install -q condacolab #-U "git+https://github.com/mayankchetan/condacolab.git@py312update"
  import condacolab
  condacolab.install() #### expect a kernel restart

In [ ]:
# Global variable to check if we're in Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !git clone  https://github.com/wisdem/weis
    !git clone https://github.com/mayankchetan/weisWorkshop

    %cd weis
    !conda config --env --add pinned_packages "numpy<2.0"
    !mamba env update -n base -f environment.yml "python<3.13"
    %pip install .
    %cd ..
    %pip install "rosco==2.9.5"
    %pip install -U "git+https://github.com/NREL/ROSCO@v2.9.7"
    %pip install "orbit-nrel<1.2.2"
    %pip install gdown

else:
    try:
        import weis

    except ImportError:
        raise ImportError("Please install the weis package from https://github.com/wisdem/weis in the environment you are using to run this notebook.")

    Warning("Make sure you have cloned 'https://github.com/mayankchetan/weisWorkshop-Private' repository and are running the notebook from the root directory of the repository.")


In [ ]:
# Downloading the precomputed data from Google Drive & extracting
# Drive link for non Google Colab users: https://drive.google.com/file/d/145BH393nzpT2msCVsHSBxR3Qy8KdpddI/view?usp=sharing
if IN_COLAB:
    %gdown --id 145BH393nzpT2msCVsHSBxR3Qy8KdpddI -O /content/weisWorkshop/archive_weisWorkshop_OCT25.tar.gz
    %tar -xvzf /content/weisWorkshop/archive_weisWorkshop_OCT25.tar.gz -C /content/weisWorkshop/
else:
    print("Please download the precomputed data from the link given in the notebook and extract it in the root directory of the repository.")
    print("you can use `tar -xvzf <downloaded_file_name>` to extract the files.")

# Introduction
 > Write some info

# Objectives
You are assigned to design and optimize a 20MW Floating wind turbine with the following constriants.
1. test
2. test
3. test

# The Approach 
1. Start with the IEA 22MW folating turbine. Establish a baseline 
2. Modify the rotor, try aero, structural and aero + structural optimization
3. Optimize the controller to the updated rotor
4. Optimize the substructure uisng RAFT
5. Continue optimization using OpenFAST & OpenFAST+Controller
6. Conduct a full DLC analysis

In [ ]:
if IN_COLAB:
    # Now we try loading the weis_workshop module again
    sys.path.append('/content/weisWorkshop/jupyterNotebooks')

# Importing nesscary libraries.
import numpy as np
import matplotlib.pyplot as plt
from weis import weis_main
from wisdem import run_wisdem
import os
import copy
import pandas as pd

from weis_workshop import *

# workshop repository folder
if IN_COLAB:
    WORKSHOP_REPO = '/content/weisWorkshop'
else:
    WORKSHOP_REPO = os.path.join(os.getcwd(), '..')

## Stage 0: Establishing a baseline

We begin with establishing a baseline using an exisitng wind turbine model. Since we are desgining an 20MW model, lets start with the IEA 22MW turbine. 

In [ ]:
# Setting up file paths
stageFolder = "stage-0-baseline"

fname_wt_input = os.path.join(WORKSHOP_REPO, "source", "IEA-22-280-RWT_Floater.yaml")
fname_modeling_options = os.path.join(WORKSHOP_REPO, stageFolder, "stage-0-baseline_modeling.yaml")
fname_analysis_options = os.path.join(WORKSHOP_REPO, stageFolder, "stage-0-baseline_analysis.yaml")


## Lets Open the input files and take a Look at them.
### Google Colab Users:
1. Go to the file browser on the left
2. Navigate to the cloned repository
3. Open the YAML files to examine the configuration

## Turibne geometry file
This file defines the trubine using the WindIO format. Currently we are using the v2.0 of WindIO. Lets take a look at the high level parts of the file.

```yaml
name: IEA 22MW Offshore Wind Turbine in Floating Configuration
assembly:
    turbine_class: I
    turbulence_class: B
    drivetrain: direct_drive
    rotor_orientation: Upwind
    number_of_blades: 3
    hub_height: 170.0
    rotor_diameter: 284.0
    rated_power: 22000000.0
    lifetime: 25.0
components:
  blade:
    outer_shape_bem:
      reference_axis:
        x: { grid: [...], values: [...] }
        y: { grid: [...], values: [...] }
        z: { grid: [...], values: [...] }
      chord: { grid: [...], values: [...] }
      twist: { grid: [...], values: [...] }
    internal_structure_2d_fem:
      reference_axis: [...]
      layers: [...]
      webs: [...]
  hub:
    diameter: 7.94  # m
    cone_angle: -2.5  # degrees
    
  nacelle:
    drivetrain: [...]
    transformer: [...]
    
  tower:
    outer_shape_bem: [...]
    internal_structure_2d_fem: [...]
    
  mooring:
    nodes: [...]
    lines: [...]
airfoils: [...]
materials: [...]
controls: [...]
environment: [...]
bos: [...]
costs: [...]
```


## analysis and modeling options
These files define the analysis and modeling options for the WISDEM/WEIS

### Analysis options
```yaml
general:
    folder_output: outputs/
    fname_output: stage-0-baseline
```
### Modeling options
```yaml
General:
    verbosity: False

WISDEM:
    RotorSE:
        flag: True
    TowerSE:
        flag: True
    DriveSE:
        flag: True
    FloatingSE:
        flag: True
    BOS:
        flag: True

OpenFAST: # Options for WEIS fidelity level 3 = nonlinear time domain
    flag: False

RAFT:
    flag: False

ROSCO:
    flag: True
    tuning_yaml: ../source/iea22_rosco.yaml
```


In [ ]:
# lets overide a few things so that we dont distrupt the existing results.
analysis_override = {}
analysis_override['general'] = {}
analysis_override['general']['folder_output'] = 'outputsCh1'


iea22mw, modeling_options, opt_options = weis_main(fname_wt_input, 
                                                 fname_modeling_options, 
                                                 fname_analysis_options,
                                                 analysis_override=analysis_override,
                                                 test_run=False
                                                 )

In [ ]:
# Lets analyize the baseline design!!
baseline_turb = pd.read_csv(os.path.join(WORKSHOP_REPO, stageFolder, "outputsCh1/stage-0-baseline.csv"), index_col=0)
# Open the CSV file shown above so explore the various variables output from WISDEM/WEIS

print(f'there are {len(baseline_turb.index)} variables in the baseline design:')

baseline_turb.index 

As you can see from the listing the variables in the CSV files, WEIS outputs a lot of information about the wind turbine model/run.

Lets look at a few of them to help us understand the baseline design.
- rotorse.rp.powercurve.V --> Wind speed (m/s)
- rotorse.rp.powercurve.Cp_aero --> Power Coefficient
- rotorse.rp.powercurve.Ct_aero --> Thrust Coefficient
- rotorse.rp.powercurve.P --> Power (W)
- rotorse.rp.powercurve.T --> Thrust (N)
- rotorse.rp.powercurve.pitch --> Pitch Angle (deg)

In [ ]:
plot_comparison_plot(baseline_turb, label='IEA22MW')

## This looks good! or does it?
- At a first glance the power curve looks good. But something seems off. Any guesses?
- Cp for the IEA 22MW is ~0.49, but the plot above shows a Cp of ~0.45. Why is that?
- The IEA 22MW turbine is higly flexible and is torsionally compilant. The performace above uses CCBlade in the back end that does not account for blade flexibility. To be able to capture the correct performance we need to use a higher fidelity blade model like BeamDyn in OpenFAST. But for a an initial design and optimization pass like we are doing today, this is a good start.



## Converting the IEA 22MW to a 20MW
- We start by modifying the IEA 22MW turbine input geometry file.
- To start, we can take the equation for power of a rotor.

 $P = \frac{1}{2} \rho A V^3 C_p$

- We can assume, $\rho$ $V$ and $C_p$ to be constant. So for a 20MW turbine, we have

 $D_{20MW} = \sqrt{ 284.0^2 * \frac{20}{22}}  = 270.0$

 - Now we make a copy of the IEA 22MW turbine geometry file and modify the rotor diameter and power rating.

``` yaml
assembly:
    turbine_class: I
    turbulence_class: B
    drivetrain: direct_drive
    rotor_orientation: Upwind
    number_of_blades: 3
    hub_height: 170.0
    rotor_diameter: 270.0 # 284.0
    rated_power: 20000000.0 # 22000000.0
    lifetime: 25.0
```


# Optimizing the rotor!

Now that we have a baseline design, and the new 20MW geometry file, we can start optimizing the rotor. 

To have a simplified approach, we can use WISDEM to optimize the rotor, instead of using a more complex approach using OpenFAST+Controller. This approach will give us first pass quick and dirty optimization. This is also how we use multiple tools in the optimization process in a systematic way, and not setup a single large complex optimization problem that might be hard to solve.

In [ ]:
# Setting up file paths, and defining the WEIS overide for this stage
stageFolder = "stage-1-aeroStruct"

fname_wt_input = os.path.join(WORKSHOP_REPO, "source", "new-20-270-RWT_Floater.yaml")
fname_modeling_options = os.path.join(WORKSHOP_REPO, stageFolder, "stage-1-aeroStruct_analysis.yaml")
fname_analysis_options = os.path.join(WORKSHOP_REPO, stageFolder, "stage-1-aeroStruct_modeling.yaml")

# We dont have analysis, and modeling options overrides in WISDEM
# analysis_override = {}
# analysis_override['general'] = {}
# analysis_override['general']['folder_output'] = 'outputsCh1'
# analysis_override['driver'] = {}
# analysis_override['driver']['max_iter'] = 1 # only run one iteration for this demo, we have precomputed the solutions for you :)

## Lets Open the input files and take a Look at them.
### Google Colab Users:
1. Go to the file browser on the left
2. Navigate to the cloned repository
3. Open the YAML files to examine the configuration

## Analysis options:
```yaml

design_variables:
 
    blade:
        aero_shape:
            twist:
                flag: False # Flag to optimize the twist
                inverse: True # Flag to determine twist from a desired angle of attack
                inverse_target: 'max_efficiency' # Twist generates angles of attack # corresponding to 'max_efficiency' or 'stall_margin'
                n_opt: 10 # Number of control points along blade span. During inverse design, # twist is smoothened with a spline with these
                max_decrease: 0.08722222222222221 # Maximum decrease for the twist in [rad] at the n_opt locations. Only used if flag is set to True
                max_increase: 0.08722222222222221 # Maximum increase for the twist in [rad] at the n_opt locations. Only used if flag is set to True
                index_start: 2 # Lock the first two DVs from blade root
                index_end: 10 # All DVs close to blade tip are active

            chord:
                flag: True # Flag to optimize the chord
                n_opt: 10 # Number of control points along blade span
                max_decrease: 0.4 # Minimum multiplicative gain on existing chord at the n_opt locations
                max_increase: 1.5 # Maximum multiplicative gain on existing chord at the n_opt locations
                index_start: 1 # Lock the first DV from blade root
                index_end: 9 # All DVs close to blade tip are active
            af_positions:
                flag: True
                af_start: 4
        structure:
            - layer_name: spar_cap_ss
              n_opt: 8               # Number of control points along blade span
              max_decrease: 0.2      # Maximum nondimensional decrease at the n_opt locations
              max_increase: 5.       # Maximum nondimensional increase at the n_opt locations
              index_start: 1         # Lock the first DV from blade root
              index_end: 7           # The last DV at blade tip is locked
            - layer_name: spar_cap_ps
              n_opt: 8               # Number of control points along blade span
              max_decrease: 0.2      # Maximum nondimensional decrease at the n_opt locations
              max_increase: 5.       # Maximum nondimensional increase at the n_opt locations
              index_start: 1         # Lock the first DV from blade root
              index_end: 7           # The last DV at blade tip is locked
            - layer_name: te_reinforcement_ss
              n_opt: 8               # Number of control points along blade span
              max_decrease: 0.2      # Maximum nondimensional decrease at the n_opt locations
              max_increase: 5.       # Maximum nondimensional increase at the n_opt locations
              index_start: 1         # Lock the first DV from blade root
              index_end: 7           # The last DV at blade tip is locked
            - layer_name: te_reinforcement_ps
              n_opt: 8               # Number of control points along blade span
              max_decrease: 0.2      # Maximum nondimensional decrease at the n_opt locations
              max_increase: 5.       # Maximum nondimensional increase at the n_opt locations
              index_start: 1         # Lock the first DV from blade root
              index_end: 7           # The last DV at blade tip is locked

# figure of merit
merit_figure: LCOE

constraints:
    blade:
        stall:
            flag: False # Constraint on minimum stall margin
            margin: 0.12 # Value of minimum stall margin in [rad]
        chord:
            flag: True # Constraint max chord to its default value (4.75 m)
            max: 8. # Max chord value
        chord_slope:
            flag: True # Constraint to enforce monothonically decreasing chord after max chord
        root_circle_diameter:
            flag: False # Constraint for the minimum blade root circle diameter
            max_ratio: 1.2 # The recommended value can be up to 20% larger than the actual
        moment_coefficient:
            flag: False # Constraint on maximum blade root flapwise moment coefficient
            max: 0.20 # Lowering this value will lead to low induction rotors
        strains_spar_cap_ss:
            flag: True         # Flag to impose constraints on maximum strains (absolute value) in the spar cap on the blade suction side
            max:    3500.e-6   # Value of maximum strains [-]
            index_start: 1     # Do not enforce constraint at the first station from blade root of the n_opt from spar_cap_ss
            index_end: 7       # Do not enforce constraint at the last station at blade tip of the n_opt from spar_cap_ss
        strains_spar_cap_ps:
            flag: True         # Flag to impose constraints on maximum strains (absolute value) in the spar cap on the blade pressure side
            max:    3500.e-6   # Value of maximum strains [-]
            index_start: 1     # Do not enforce constraint at the first station from blade root of the n_opt from spar_cap_ps
            index_end: 7       # Do not enforce constraint at the last station at blade tip of the n_opt from spar_cap_ps      
        tip_deflection:
            flag: True
            margin: 1.4175

driver:
    optimization:
        flag: True          # Flag to enable optimization
        tol: 1.e-6          # Optimality tolerance
        max_iter: 1000      # Maximum number of iterations (SLSQP)
        solver: SLSQP       # Optimization solver. Other options are 'SLSQP' - 'CONMIN'
        step_size: 1.e-3    # Step size for finite differencing
        form: central       # Finite differencing mode, either forward or central

recorder:
    flag: True              # Flag to activate OpenMDAO recorder
    file_name: log_opt.sql  # Name of OpenMDAO recorder


```

## Modeling Options:

```yaml
General:
    verbosity: True  # When set to True, the code prints to screen many infos


WISDEM:
    RotorSE:
        flag: True
        spar_cap_ss: spar_cap_ss
        spar_cap_ps: spar_cap_ps

    DriveSE:
        flag: False
    TowerSE:
        flag: True
    FloatingSE:
        flag: False
    BOS:
        flag: False

```


# Before running the next cell, make sure you have done the following:
1. Update the analysis option output path to `outputsCh1`
2. Update the max_iter to 1 --> only run one iteration for this demo, we have precomputed the solutions for you :)

In [ ]:
# input("Have you updated the analysis options file? Press Enter to continue...")

wt_opt, modeling_options, analysis_options = run_wisdem(
        fname_wt_input, fname_modeling_options, fname_analysis_options
    )


## Now lets analyze the results!

1. We have run the optimization for you! The results are stored in the `outputs` folder. Lets load the results and plot them.
2. Additionally, we ran the case without any optimization to compare the 'before and after' of the optimization.

In [ ]:
baseline_turb = pd.read_csv(os.path.join(WORKSHOP_REPO, stageFolder,"outputs_nonOpti/stage-1-aeroStruct-NonOptimized.csv"), index_col=0)
optimized_turb = pd.read_csv(os.path.join(WORKSHOP_REPO, stageFolder,"outputs_preCompute/stage-1-aeroStruct-aero_analysis.csv"), index_col=0)

plot_rotor_comparison(baseline_turb, optimized_turb)

## Lets talk about caveats!
- As discussed earlier, the optimization is done using CCBlade in the back end. So the performance numbers are not exact. But the trends are correct.
- If you'd like to capture a more accurate performance, you'll need to use WEIS with OpenFAST+BeamDyn, which can be very computationally expensive.

## Hows the convergance of the optimization?
- The optimization converged in 44 iterations.
- We can use the log_opt.sql that logs the optimization history to plot the convergance.

In [ ]:
rec_data = load_OMsql(os.path.join(WORKSHOP_REPO, stageFolder,"outputs_preCompute/log_opt.sql")) # not sure what supresses the output here
plot_convergence(rec_data, ['financese.lcoe', 'tcons.tip_deflection_ratio', 'blade.opt_var.af_position', 'blade.opt_var.chord_opt', 'blade.pa.max_chord_constr',
                            'rotorse.rs.constr.constr_max_strainL_spar', 'rotorse.rs.constr.constr_max_strainU_spar'])

# We have a new rotor!!

## Things we accomplished in this session!
- We started with an existing turbine model, the IEA 22MW floating wind turbine and established a baseline
- An overview of the WEIS/WISDEM input files and their structure
- We ran the IEA 22MW turbine to establish a baseline, and plotted the results from the CSV file
- We modified the IEA 22MW turbine to a 20MW turbine by changing the rotor diameter and rated power
- We optimized the rotor using WISDEM
- We inspected the optimization log using the log_opt.sql file, and plotted the convergence
- We compared the unoptimized and optimized 20MW rotors.